In [1]:
import os
import scanpy as sc
import numpy as np
import multiprocessing

from sklearn.cluster import KMeans
from pyclustering.cluster.clarans import clarans

/Users/antonogorodnikov/anaconda3/envs/scito/lib/python3.8/site-packages/anndata/core/anndata.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
path = '/Users/antonogorodnikov/Documents/Work/DataSci/SCITO-seq/tests/100k_pbmc_filtered_feature_bc_matrix.h5'

In [3]:
batchid_string='barcode'

In [4]:
adata = sc.read_10x_h5(path, gex_only=False)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [8]:
n_clust=None
batches = adata.var.index.str.extract(r'(%s\d+)'%batchid_string).iloc[:,0].dropna().unique()
nClust = n_clust if n_clust != None else len(batches)+1

array(['barcode1', 'barcode2', 'barcode3', 'barcode4', 'barcode5',
       'barcode6', 'barcode7', 'barcode8', 'barcode9', 'barcode10'],
      dtype=object)

In [9]:
ab_adata = adata[:,adata.var.index.str.contains(r'(%s\d+)'%batchid_string)]

/Users/antonogorodnikov/anaconda3/envs/scito/lib/python3.8/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [10]:
ab_adata

View of AnnData object with n_obs × n_vars = 35371 × 280 
    var: 'gene_ids', 'feature_types', 'genome'

In [38]:
len(ab_adata.X.todense().tolist()[0])

280

In [6]:
len(np.array(adata.X.todense())[0])

33818

In [7]:
adata

AnnData object with n_obs × n_vars = 35371 × 33818 
    var: 'gene_ids', 'feature_types', 'genome'

In [12]:
len(batches)+1

11

In [43]:
ab_adata = adata[:,adata.var.index.str.contains(r'(%s\d+)'%batchid_string)]

/Users/antonogorodnikov/anaconda3/envs/scito/lib/python3.8/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [44]:
n_clust=None
n_init=100
kfunc="clarans"
maxneighbor=100
seed=2

In [17]:
cell_adata_p = adata[:,-280:]


a=np.sum(adata[:,:28].X, axis=1)
b=np.sum(adata[:,28:56].X, axis=1)
c=np.sum(adata[:,56:84].X, axis=1)
d=np.sum(adata[:,84:112].X, axis=1)
e=np.sum(adata[:,112:140].X, axis=1)
f=np.sum(adata[:,140:168].X,axis=1)
g=np.sum(adata[:,168:196].X,axis=1)
h=np.sum(adata[:,196:224].X,axis=1)
i=np.sum(adata[:,224:252].X,axis=1)
j=np.sum(adata[:,252:280].X,axis=1)


In [26]:
ab_hashtag.shape

(35371, 10)

In [19]:
import pandas as pd


In [42]:
t = (cell_adata_p.copy().T).X.toarray()
pd.DataFrame(data=t).to_csv('ab_raw_200K_matix.csv', columns=None)



Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [41]:
t

array([[  2.,   3.,   3., ..., 387., 284.,   2.],
       [  2.,   2.,   2., ...,   0.,   1.,   1.],
       [  0.,   1.,   1., ...,   4.,  14.,   0.],
       ...,
       [  1.,   1.,   0., ...,   0.,   0.,   3.],
       [  1.,   1.,   0., ...,   2.,   0.,   1.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32)

In [43]:
ab_hashtag= np.concatenate((a,b,c,d,e,f,g,h,i,j), axis=1)
ab_hashtag_df = pd.DataFrame(ab_hashtag.T)

ab_hashtag_df.to_csv("ab_merge_batch_200K_matrix.txt",header=False)



In [44]:
ab_hashtag_df

,0,1,2,3,4,5,6,7,8,9,...,35361,35362,35363,35364,35365,35366,35367,35368,35369,35370
0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0
2,2.0,3.0,1.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,...,0.0,2.0,0.0,1.0,4.0,0.0,0.0,2.0,0.0,3.0
3,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0
5,7.0,7.0,7.0,1.0,9.0,6.0,5.0,7.0,3.0,0.0,...,4.0,14.0,0.0,11.0,4.0,3.0,10.0,13.0,2.0,6.0
6,3.0,2.0,0.0,0.0,0.0,3.0,1.0,2.0,1.0,0.0,...,3.0,2.0,0.0,1.0,2.0,2.0,0.0,4.0,0.0,4.0
7,0.0,0.0,1.0,1.0,2.0,0.0,2.0,2.0,1.0,0.0,...,3.0,2.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0
8,3.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,1.0,3.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,4.0
9,5.0,1.0,0.0,0.0,1.0,0.0,3.0,5.0,0.0,0.0,...,2.0,3.0,0.0,0.0,1.0,1.0,1.0,5.0,2.0,9.0
